In [1]:
import pandas as pd
import os
import shutil

In [2]:
nuevo_nombre_carpeta = 'all_data_2005xstation_15_05_2024'  

# Section 1

Primero unimos los nuevos datos a los datos ya guardados en su respectiva estación

#Valores hasta 15 de mayo 2024
AJM.csv AJM.csv
108873 1064 109937
BJU.csv BJU.csv
121613 1065 122678
CAM.csv CAM.csv
217118 1014 218132
CCA.csv CCA.csv
133365 1053 134418
CUA.csv CUA.csv
228566 1042 229608
GAM.csv GAM.csv
109108 1064 110172
MGH.csv MGH.csv
116118 1063 117181
NEZ.csv NEZ.csv
194751 1058 195809
PED.csv PED.csv
254680 992 255672
SAC.csv SAC.csv
56018 1066 57084
SAG.csv SAG.csv
247532 931 248463
SFE.csv SFE.csv
134231 0 134231
TAH.csv TAH.csv
228177 1052 229229
TLI.csv TLI.csv
226817 1058 227875
VIF.csv VIF.csv
234964 1010 235974

In [2]:

# Tener cuidado, descomentar y comentar para ejecutar SOLO 1 vez
"""
dir = "all_data_2005xstation_31_03_2024"
dirNewData = "datos_por_estacion"
filesPREV = os.listdir(dir)
filesNEW = os.listdir(dirNewData)
for file_name in filesPREV:
  for file_nameNEW in filesNEW:
    if file_name == file_nameNEW:
      print(file_name, file_nameNEW)
      file_pathPREV = os.path.join(dir,file_name)
      datasetPREV = pd.read_csv(file_pathPREV)
      file_pathNEWdata = os.path.join(dirNewData,file_name)
      datasetNEWdata = pd.read_csv(file_pathNEWdata)
      new_df = pd.concat([datasetPREV, datasetNEWdata])
      new_df.to_csv(file_pathPREV, index=False)
      print(len(datasetPREV), len(datasetNEWdata), len(new_df))
"""

AJM.csv AJM.csv
108873 1064 109937
BJU.csv BJU.csv
121613 1065 122678
CAM.csv CAM.csv
217118 1014 218132
CCA.csv CCA.csv
133365 1053 134418
CUA.csv CUA.csv
228566 1042 229608
GAM.csv GAM.csv
109108 1064 110172
MGH.csv MGH.csv
116118 1063 117181
NEZ.csv NEZ.csv
194751 1058 195809
PED.csv PED.csv
254680 992 255672
SAC.csv SAC.csv
56018 1066 57084
SAG.csv SAG.csv
247532 931 248463
SFE.csv SFE.csv
134231 0 134231
TAH.csv TAH.csv
228177 1052 229229
TLI.csv TLI.csv
226817 1058 227875
VIF.csv VIF.csv
234964 1010 235974


Se renombra la carpeta con la fecha de los últimos datos

In [6]:
# Renombrar la carpeta
os.rename(dir, nuevo_nombre_carpeta)

Luego los segmentamos en 80%, 15% y 5% para entrenamiento, validación y prueba respectivamente
(train valid test)

In [16]:
dir = nuevo_nombre_carpeta
new_dirTest = "prueba_datos_por_estacion"
new_dirTrain = "entrenamiento_datos_por_estacion"
new_dirVal = "validacion_datos_por_estacion"

if not os.path.exists(new_dirTest):
    os.makedirs(new_dirTest)
    
if not os.path.exists(new_dirTrain):
    os.makedirs(new_dirTrain)
    
if not os.path.exists(new_dirVal):
    os.makedirs(new_dirVal)

info_datasets = []
columnas = ["file_name", "len_dataset",".8dataset", "len_trainSet", "train%", "len_testSet", "test%", "lenValSet", "val%", "%datosFaltantes", "%dastosFaltantesTrain", "%dastosFaltantesTest", "%dastosFaltantesVal"]
files = os.listdir(dir)
for file_name in files:
  file_path = os.path.join(dir,file_name)
  dataset = pd.read_csv(file_path)
  dataset[['year', 'month', 'day']] = dataset['date'].str.split('/', expand=True)
  dataset["hour"] = dataset["day"]
  dataset[["day", "hour"]] = dataset["day"].str.split(' ', expand=True)
  dataset[["hour","minute"]] = dataset["hour"].str.split(':', expand=True)
  train_procentage = round(len(dataset)*.8)
  validation_porcentage = round(len(dataset)*.15)
  test_porcentage = round(len(dataset)*.05)
  train_set = dataset[0:train_procentage]
  test_set = dataset[train_procentage:train_procentage+test_porcentage]
  val_set = dataset[train_procentage+test_porcentage:len(dataset)]
  porcentaje_total_vacios = round(dataset.isna().mean().mean() * 100,2)
  porc_vacios_train = round(train_set.isna().mean().mean() * 100,2)
  porc_vacios_test = round(test_set.isna().mean().mean() * 100,2)
  porc_vacios_val = round(val_set.isna().mean().mean() * 100,2)
  info_datasets.append([file_name, len(dataset), round(len(dataset)*.8), len(train_set), round(len(train_set)*100/len(dataset)), len(test_set), round(len(test_set)*100/len(dataset)),len(val_set), round((len(val_set)*100/len(dataset)),2), porcentaje_total_vacios, porc_vacios_train, porc_vacios_test, porc_vacios_val])
  new_nameTrain = os.path.join(new_dirTrain, file_name)
  new_nameTest = os.path.join(new_dirTest, file_name)
  new_nameVal = os.path.join(new_dirVal, file_name)
  train_set.to_csv(new_nameTrain, index=False)
  test_set.to_csv(new_nameTest, index=False)
  val_set.to_csv(new_nameVal, index=False)
  print(new_nameTrain, len(dataset), round(len(dataset)*.8), len(train_set), round(len(train_set)*100/len(dataset)), len(test_set), round(len(test_set)*100/len(dataset)),len(val_set), round((len(val_set)*100/len(dataset)),2), porcentaje_total_vacios, porc_vacios_train, porc_vacios_test, porc_vacios_val)

info_name = "info_split_datasets.csv"
info_datasets = pd.DataFrame(info_datasets, columns=columnas)
info_datasets.to_csv(info_name, index=False)

entrenamiento_datos_por_estacion\AJM.csv 109937 87950 87950 80 5497 5 16490 15.0 25.45 25.77 40.44 18.77
entrenamiento_datos_por_estacion\BJU.csv 122678 98142 98142 80 6134 5 18402 15.0 32.91 33.35 44.31 26.75
entrenamiento_datos_por_estacion\CAM.csv 218132 174506 174506 80 10907 5 32719 15.0 40.68 41.91 33.63 36.42
entrenamiento_datos_por_estacion\CCA.csv 134418 107534 107534 80 6721 5 20163 15.0 36.57 36.68 36.47 36.07
entrenamiento_datos_por_estacion\CUA.csv 229608 183686 183686 80 11480 5 34442 15.0 32.69 33.71 31.45 27.68
entrenamiento_datos_por_estacion\GAM.csv 110172 88138 88138 80 5509 5 16525 15.0 39.18 38.8 52.39 36.78
entrenamiento_datos_por_estacion\MGH.csv 117181 93745 93745 80 5859 5 17577 15.0 26.87 25.98 46.37 25.14
entrenamiento_datos_por_estacion\NEZ.csv 195809 156647 156647 80 9790 5 29372 15.0 32.99 34.12 33.42 26.78
entrenamiento_datos_por_estacion\PED.csv 255672 204538 204538 80 12784 5 38350 15.0 25.76 26.22 25.09 23.53
entrenamiento_datos_por_estacion\SAC.csv 57

Las carpetas con los datos x estación ya segmentados se encuentran en entrenamiento / prueba / y validación

# Section 2

Se crean nuevas carpetas con archivos sin datos faltantes

In [17]:
dir = "entrenamiento_datos_por_estacion"
dir_sinNaN = "entrenamiento_datos_por_estacion_sin_NaN"
if not os.path.exists(dir_sinNaN ):
    os.makedirs(dir_sinNaN )
files = os.listdir(dir)
for file_name in files:
  file_path = os.path.join(dir,file_name)
  print(file_path)
  dataset = pd.read_csv(file_path)
  dataset = dataset.dropna(axis=1, how='all')
  dataset = dataset.dropna(axis=0)
  df = dataset[["date", "year", "month", "day", "hour", "minute"]]
  dataset = dataset.drop(["date", "year", "month", "day", "hour", "minute"], axis=1)
  for column in dataset.columns:
    if dataset[column].nunique() == 1:
      dataset = dataset.drop(columns=[column], axis=1)
  dataset.insert(0, "date", df["date"])
  df = df.drop(["date"], axis=1)
  dataset[["year", "month", "day", "hour", "minute"]] = df
  name = os.path.join(dir_sinNaN, file_name)
  dataset.to_csv(name, index=False)

dir = "validacion_datos_por_estacion"
dir_sinNaN = "validacion_datos_por_estacion_sin_NaN"
if not os.path.exists(dir_sinNaN):
    os.makedirs(dir_sinNaN )
files = os.listdir(dir)
for file_name in files:
  file_path = os.path.join(dir,file_name)
  print(file_path)
  dataset = pd.read_csv(file_path)
  dataset = dataset.dropna(axis=1, how='all')
  dataset = dataset.dropna(axis=0)
  df = dataset[["date", "year", "month", "day", "hour", "minute"]]
  dataset = dataset.drop(["date", "year", "month", "day", "hour", "minute"], axis=1)
  for column in dataset.columns:
    if dataset[column].nunique() == 1:
      dataset = dataset.drop(columns=[column], axis=1)
  dataset.insert(0, "date", df["date"])
  df = df.drop(["date"], axis=1)
  dataset[["year", "month", "day", "hour", "minute"]] = df
  name = os.path.join(dir_sinNaN, file_name)
  dataset.to_csv(name, index=False)


dir = "prueba_datos_por_estacion"
dir_sinNaN = "prueba_datos_por_estacion_sin_NaN"
if not os.path.exists(dir_sinNaN):
    os.makedirs(dir_sinNaN )
files = os.listdir(dir)
for file_name in files:
  file_path = os.path.join(dir,file_name)
  print(file_path)
  dataset = pd.read_csv(file_path)
  dataset = dataset.dropna(axis=1, how='all')
  dataset = dataset.dropna(axis=0)
  df = dataset[["date", "year", "month", "day", "hour", "minute"]]
  dataset = dataset.drop(["date", "year", "month", "day", "hour", "minute"], axis=1)
  for column in dataset.columns:
    if dataset[column].nunique() == 1:
      dataset = dataset.drop(columns=[column], axis=1)
  dataset.insert(0, "date", df["date"])
  df = df.drop(["date"], axis=1)
  dataset[["year", "month", "day", "hour", "minute"]] = df
  name = os.path.join(dir_sinNaN, file_name)
  dataset.to_csv(name, index=False)

entrenamiento_datos_por_estacion\AJM.csv
entrenamiento_datos_por_estacion\BJU.csv
entrenamiento_datos_por_estacion\CAM.csv
entrenamiento_datos_por_estacion\CCA.csv
entrenamiento_datos_por_estacion\CUA.csv
entrenamiento_datos_por_estacion\GAM.csv
entrenamiento_datos_por_estacion\MGH.csv
entrenamiento_datos_por_estacion\NEZ.csv
entrenamiento_datos_por_estacion\PED.csv
entrenamiento_datos_por_estacion\SAC.csv
entrenamiento_datos_por_estacion\SAG.csv
entrenamiento_datos_por_estacion\SFE.csv
entrenamiento_datos_por_estacion\TAH.csv
entrenamiento_datos_por_estacion\TLI.csv
entrenamiento_datos_por_estacion\VIF.csv
validacion_datos_por_estacion\AJM.csv
validacion_datos_por_estacion\BJU.csv
validacion_datos_por_estacion\CAM.csv
validacion_datos_por_estacion\CCA.csv
validacion_datos_por_estacion\CUA.csv
validacion_datos_por_estacion\GAM.csv
validacion_datos_por_estacion\MGH.csv
validacion_datos_por_estacion\NEZ.csv
validacion_datos_por_estacion\PED.csv
validacion_datos_por_estacion\SAC.csv
valid